In [ ]:
ZROK_TOKEN = "zrok_token_here"

import os
import IPython.display

if not os.path.exists("/tmp/python/done"):
    print("SETUP WILL TAKE A FEW MINUTES...\n")
    %rm -rf /tmp/*
    %cd /tmp
    !curl -L "https://github.com/indygreg/python-build-standalone/releases/download/20230726/cpython-3.10.12+20230726-x86_64_v3-unknown-linux-gnu-install_only.tar.gz" -o "python.tar.gz" &&\
        tar -xf "python.tar.gz" &&\
        rm "python.tar.gz"

    if not os.path.exists("/tmp/python/bin"):
        IPython.display.clear_output()
        raise Exception("FAILED TO DOWNLOAD PYTHON")

    %cd /tmp/python/bin
    !./pip install --no-cache-dir -U pip &&\
        ./pip install --no-cache-dir torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 &&\
        ./pip install --no-cache-dir diffusers==0.21.0 einops==0.6.1 k_diffusion==0.0.15 lark==1.1.5 safetensors==0.3.1 tqdm==4.65.0 transformers==4.36.2 spandrel==0.1.8 timm==0.9.2 tomesd==0.1.3 segment-anything==1.0 geffnet==1.0.2 toml==0.10.2 voluptuous==0.13.1 websockets==11.0.3 bson==0.5.10 mega.py==1.0.8 pytorch-lightning==2.0.2 cryptography lycoris-lora==2.3.0.dev4 ultralytics==8.2.3 &&\
        ./pip uninstall -y opencv-python && ./pip install opencv-python-headless==4.9.0.80 &&\
        touch /tmp/python/done

    if not os.path.exists("/tmp/python/done"):
        IPython.display.clear_output()
        raise Exception("FAILED TO DOWNLOAD ENVIRONMENT")

    IPython.display.clear_output()
    print("DONE")

%cd /home/studio-lab-user
from IPython.core.display import display, HTML
import sys
import random
import string
import urllib.parse
import subprocess
import threading
import time
import re
import socket
%env MPLBACKEND=notebook

if not os.path.exists("/home/studio-lab-user/sd-inference-server"):
    !git clone https://github.com/arenasys/sd-inference-server.git
    %cd /home/studio-lab-user/sd-inference-server
    !git clone https://github.com/arenasys/sd-scripts.git training
    !wget https://github.com/openziti/zrok/releases/download/v0.4.25/zrok_0.4.25_linux_amd64.tar.gz
    !tar -xzf zrok_0.4.25_linux_amd64.tar.gz

%cd /home/studio-lab-user/sd-inference-server
!git pull

model_folder = "/home/studio-lab-user/sd-inference-server/models"

if not sys.path[0] == "/home/studio-lab-user/sd-inference-server/":
    sys.path.insert(0, "/home/studio-lab-user/sd-inference-server/")

IPython.display.clear_output()
print("STARTING...")
password = ''.join(random.SystemRandom().choice(string.ascii_letters + string.digits) for _ in range(8))

!./zrok enable "$ZROK_TOKEN"

stopping = False

def tunnel_thread():
    while True:
        time.sleep(0.1)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', 28888))
        if result == 0:
            break
        sock.close()
    time.sleep(1)

    url_pattern = re.compile(r"(?P<url>https?://[^\s]*\.zrok\.io)")
    p = subprocess.Popen(["./zrok", "share", "public", "localhost:28888", "--headless"], stdout=subprocess.DEVNULL, stderr=subprocess.PIPE, encoding="utf-8")
    for line in p.stderr:
        url_match = url_pattern.search(line)
        if url_match:
            tunnel_url = url_match.group("url")
            endpoint = tunnel_url.replace("https", "wss").strip()
            params = urllib.parse.urlencode({'endpoint': endpoint, "password": password})
            style = "background-color:rgba(0,0,0,0);font-size:var(--jp-code-font-size);margin: 0px;padding-left: var(--jp-code-padding);"
            print("ENDPOINT:", endpoint)
            display(HTML(f'<pre style="{style}">DESKTOP: <a href="{"qdiffusion:?" + params}">{"qdiffusion:?" + params}</a></pre>'))
            print("WEB:", "https://arenasys.github.io/?" + params)
            print("READY!")
            break
    while not stopping:
        time.sleep(0.1)
    print("DIE")
    p.kill()

print("PASSWORD:", password)
threading.Thread(target=tunnel_thread, daemon=True).start()
!/tmp/python/bin/python3.10 remote.py --models "$model_folder" --password "$password"
stopping = True

In [ ]:
# check storage
!df -H /home/studio-lab-user/
!du -sh /home/studio-lab-user/*
!du -sh /home/studio-lab-user/.[!.]*

In [ ]:
# to get a zrok token, first run this cell
import IPython.display
%cd ~
!wget https://github.com/openziti/zrok/releases/download/v0.4.25/zrok_0.4.25_linux_amd64.tar.gz
!tar -xzf zrok_0.4.25_linux_amd64.tar.gz
IPython.display.clear_output()
print("DONE")
print("Now open a terminal (File->New->Terminal), then run: './zrok invite'")
print("Zrok will send you an invite to the email you speicfy, create account, set password, get your token.")
print("Your token is on the final welcome screen at 'zrok enable YOUR_TOKEN'. Also at 'https://api.zrok.io/' under 'Details'.")

In [ ]:
# to update run this cell, then close this notebook tab (discarding changes), and reopen
if input("Update? [y/n]")[0].lower() == "y":
    !rm -rf /tmp/*
    !curl https://raw.githubusercontent.com/arenasys/qDiffusion/master/remote_sagemaker.ipynb -o ~/remote_sagemaker.ipynb
    print("DONE")